In [47]:
# import python dictreader
import csv
import sys
import os

# # # Generate the tag mapping
# with open('tags_1per.csv', "r") as f:
#     reader = csv.DictReader(f)
#     relationship_tags = []
#     for row in reader:
#         if row['name'] == 'Redacted':
#             continue
#         if row['type'] == 'Relationship':
#             relationship_tags.append(row)
    
#     # Write the relationship tags
#     with open('relationship_tags.csv', 'w') as f:
#         writer = csv.DictWriter(f, fieldnames=reader.fieldnames)
#         writer.writeheader()
#         for row in relationship_tags:
#             writer.writerow(row)

# # Read relationship tags into memory
# relationship_tags = {}
# with open('relationship_tags.csv', "r") as f:
#     reader = csv.DictReader(f)
#     for row in reader:
#         relationship_tags[row['id']] = row

# print(relationship_tags['29'])

# # Read the works
# with open('works_1per.csv', "r") as f:
#     reader = csv.DictReader(f)
#     all_tags = []
#     for row in reader:
#         row_tags = row['tags'].split('+')
#         relationship_row_tags = [i for i in row_tags if i in relationship_tags]


'''
Graph structure:
nodes: characters
edges: relationships (if they appear in the same work)

Iterate over the works.
Get all the relationship tags from the work.
Skip the works with more than 1 relationship tag. 
Split the relationship tag on / or & as needed to get the characters.
Keep track of a dictionary which is a mapping of (character1, character2) -> number of occurances.
Create a graph where the nodes are characters and the edges are relationships.



# Create these mappings:
# tag id -> tag
# character name 

'''

def get_pairing(slash_pairing):
    if '/' in slash_pairing and '&' in slash_pairing:
        raise ValueError(f'Relationship tag contains both / and & {slash_pairing}')
    if '/' not in slash_pairing and '&' not in slash_pairing:
        raise ValueError(f'Relationship tag contains neither / or & {slash_pairing}')
    if slash_pairing.count('/') > 1 or slash_pairing.count('&') > 1:
        # Jack Harkness/Ianto Jones/Toshiko Sato
        raise ValueError(f'Relationship tag contains more than one / or & {slash_pairing}')
    if '/' in slash_pairing:
        return [i.strip() for i in slash_pairing.split('/')]
    if '&' in slash_pairing:
        return [i.strip() for i in slash_pairing.split('&')]

from collections import defaultdict

with open('tags_1per.csv', "r") as f:
    reader = csv.DictReader(f)
    pairing_id_to_count = defaultdict(int)
    pairing_id_to_row = {}
    for ind, row in enumerate(reader):
        if row['name'] == 'Redacted' or row['type'] != 'Relationship':
            continue
        canonical = row['canonical']
        # print(row)
        if canonical == 'true':
            pairing_id_to_row[row['id']] = row
            pairing_id_to_count[row['id']] += int(row['cached_count'])
        else:
            pairing_id_to_count[row['merger_id']] += int(row['cached_count'])
        # if ind > 50:
        #     break

In [48]:
import networkx as nx

G = nx.Graph()
for pairing_id, count in pairing_id_to_count.items():
    if pairing_id not in pairing_id_to_row:
        continue
    relationship_name = pairing_id_to_row[pairing_id]['name']
    try:
        characters = get_pairing(relationship_name)
    except ValueError as e:
        # Print
        # print(e)
        continue
    character1, character2 = characters
    # TODO Characters can have the same name
    G.add_node(character1)
    G.add_node(character2)
    G.add_edge(character1, character2, weight=count)
print(G.number_of_nodes())
print(G.number_of_edges())
print(G['Dean Winchester'])
print(G['Sam Winchester'])    


13495
15445
{'Sam Winchester': {'weight': 11301}, 'John Winchester': {'weight': 507}, 'Cassie Robinson': {'weight': 313}, 'Jared Padalecki': {'weight': 57}, 'Bela Talbot': {'weight': 190}, 'Other(s)': {'weight': 936}, 'Castiel': {'weight': 10590}, 'Ruby': {'weight': 86}, 'Jo Harvelle': {'weight': 781}, 'Bobby Singer': {'weight': 40}, 'Claire Littleton': {'weight': 0}, 'Paul Callan': {'weight': 11}, 'Shane McCutcheon': {'weight': 0}, 'Willow Rosenberg': {'weight': 5}, 'Ellen Harvelle': {'weight': 53}, 'Luna Lovegood': {'weight': 3}, 'Faith Lehane': {'weight': 107}, 'Alastair': {'weight': 356}, 'Zoë Washburne': {'weight': 0}, 'Christian Kane': {'weight': 3}, 'Impala': {'weight': 179}, 'Anna Milton': {'weight': 286}, 'Hannibal King': {'weight': 2}, 'John Sheppard': {'weight': 17}, 'Hermione Granger': {'weight': 91}, 'Veronica Mars': {'weight': 11}, 'River Tam': {'weight': 2}, 'Carmen Porter': {'weight': 39}, 'Jack Harkness': {'weight': 53}, 'Kaylee Frye': {'weight': 6}, 'Lucifer': {'weigh

In [49]:
import networkx as nx
from node2vec import Node2Vec

# Generate a synthetic graph (you can replace this with your own graph)
# G = nx.erdos_renyi_graph(n=100, p=0.1)

# Precompute probabilities and generate walks
# num_walks=20 is like 4.5 seconds
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=20, workers=4, weight_key='weight')

# Embed nodes
model = node2vec.fit(window=10, min_count=1, batch_words=4)

# Retrieve the embeddings for all nodes
embeddings = {node: model.wv[node] for node in G.nodes()}


Computing transition probabilities:   2%|▏         | 271/13495 [00:00<00:09, 1431.53it/s]/home/codespace/.python/current/lib/python3.12/site-packages/node2vec/node2vec.py:143: RuntimeWarning: invalid value encountered in divide
  d_graph[source][self.FIRST_TRAVEL_KEY] = first_travel_weights / first_travel_weights.sum()
Generating walks (CPU: 1):   0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Total of weights must be finite

In [ ]:
print(G.nodes())

# # Example: Print the embedding for node 0
# print("Embedding for Node 0:", embeddings[0])

# # Print cosine similarity between node 0 and node 1
# cosine_similarity = model.wv.similarity('0', '1')
# print("Cosine similarity between Node 0 and Node 1:", cosine_similarity)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
